In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [20]:
df = [ pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/" +x+ "_data.csv") for x in ['test','train']]

In [21]:
def flig_con(df):
    df[['flig','num']] = df['flight'].str.split('-', expand=True)
    df['num'] = df['num'].astype('int64')
    df.drop(['flight','id'], axis=1, inplace=True)
    
flig_con(df[1])
flig_con(df[0])

In [22]:
w = {'one':1,'zero':0,'two_or_more':2}
q = {'Economy':0,'Business':1}

In [23]:
def two_or_one(f):
    f['stops'] = f['stops'].map(w)
    f['class'] = f['class'].map(q)
    
two_or_one(df[1])
two_or_one(df[0])

In [24]:
num_r = ['stops','class','duration','days_left','num']
str_r = ['airline','source_city','departure_time','arrival_time','destination_city','flig']

In [25]:
full_pip = ColumnTransformer([
    ('num', StandardScaler(), num_r),
    ('srt',OneHotEncoder(),str_r)
])

In [26]:
y = df[1]['price'].copy()
X = df[1].drop('price',axis=1)

In [27]:
x_pred = full_pip.fit_transform(X)

##Modelni accuracy ni kuramiz qanchalik yuqoriligini

In [28]:
x_train,x_test,y_train,y_test = train_test_split(x_pred,y,test_size=0.2,random_state=42)

In [29]:
Rand_model = RandomForestRegressor()
Rand_model.fit(x_train,y_train)

RandomForestRegressor()

In [34]:
Rand_pred = Rand_model.predict(x_test)
mean_r = np.sqrt(mean_squared_error(y_test,Rand_pred))
mean_r

3317.8216212660304

In [30]:
Tree_model = DecisionTreeRegressor()
Tree_model.fit(x_train,y_train)

DecisionTreeRegressor()

In [33]:
Tree_pred = Tree_model.predict(x_test)
mean_t = np.sqrt(mean_squared_error(y_test,Tree_pred))
mean_t

4423.647229627099

In [31]:
xgb_model = XGBRegressor()
xgb_model.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [32]:
xgb_pred = xgb_model.predict(x_test)
mean_x = np.sqrt(mean_squared_error(y_test,xgb_pred))
mean_x

3390.3347843154465

Demak bizni modellarimizdan RandomForestRegressor() modelni mean absolut error i kichik ekan


In [35]:
test_x = full_pip.fit_transform(df[0])

In [40]:
umumiy_pred = Rand_model.predict(test_x)


In [42]:
d = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv")
d['price'] = umumiy_pred
d

,id,price
0,1,55335.48
1,2,62470.58
2,3,23412.42
3,4,2520.65
4,5,5754.57
...,...,...
4995,4996,59906.78
4996,4997,4445.46
4997,4998,50263.82
4998,4999,49430.51
